In [116]:
import Bio
import pandas
#from context import src
import skbio
import io
from skbio import DNA, RNA, Sequence
import sklearn as sk
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [119]:
from Bio import SeqIO
#SeqIO.parse(gb_file, "gb")
#GCA_000146045.2
#gb_file = "NC_001133"
i=1
for gb_record in SeqIO.parse("GCF_000146045.2_R64_genomic.gbff", "genbank") :
    # now do something with the record
    #for feature in gb_record.features:
    
    if i ==1:
        #print(gb_record.seq)
        values = np.array(list(gb_record.seq.strip()))
        print(values)
        # integer encode
        label_encoder = LabelEncoder()
        print(np.shape(values))
        integer_encoded = label_encoder.fit_transform(values)
        print(integer_encoded)
        # binary encode
        onehot_encoder = OneHotEncoder(sparse=True)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
        onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
        print(onehot_encoded)
    #print  (gb_record.name, gb_record.features[1])
    #print (repr(gb_record.seq))
    i+=1

['C' 'C' 'A' ... 'G' 'G' 'G']
(230218,)
[1 1 0 ... 2 2 2]
  (0, 1)	1.0
  (1, 1)	1.0
  (2, 0)	1.0
  (3, 1)	1.0
  (4, 0)	1.0
  (5, 1)	1.0
  (6, 1)	1.0
  (7, 0)	1.0
  (8, 1)	1.0
  (9, 0)	1.0
  (10, 1)	1.0
  (11, 1)	1.0
  (12, 1)	1.0
  (13, 0)	1.0
  (14, 1)	1.0
  (15, 0)	1.0
  (16, 1)	1.0
  (17, 0)	1.0
  (18, 1)	1.0
  (19, 1)	1.0
  (20, 1)	1.0
  (21, 0)	1.0
  (22, 1)	1.0
  (23, 0)	1.0
  (24, 1)	1.0
  :	:
  (230193, 2)	1.0
  (230194, 3)	1.0
  (230195, 2)	1.0
  (230196, 2)	1.0
  (230197, 3)	1.0
  (230198, 2)	1.0
  (230199, 3)	1.0
  (230200, 2)	1.0
  (230201, 2)	1.0
  (230202, 2)	1.0
  (230203, 3)	1.0
  (230204, 2)	1.0
  (230205, 3)	1.0
  (230206, 2)	1.0
  (230207, 2)	1.0
  (230208, 3)	1.0
  (230209, 2)	1.0
  (230210, 3)	1.0
  (230211, 2)	1.0
  (230212, 3)	1.0
  (230213, 2)	1.0
  (230214, 3)	1.0
  (230215, 2)	1.0
  (230216, 2)	1.0
  (230217, 2)	1.0


In [3]:

# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} Bio


Solving environment: / ^C
failed

CondaError: KeyboardInterrupt



In [103]:
#dna_seq=DNA.read(gb)
#record = SeqIO.read("GCF_000146045.2_R64_genomic.gbff", "genbank")

dna='agtctca'

In [111]:
values = np.array(list(dna.strip()))
print(values)
# integer encode
label_encoder = LabelEncoder()
print(np.shape(values))
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([np.argmax(onehot_encoded[0, :])])
print(inverted)

['a' 'g' 't' 'c' 't' 'c' 'a']
(7,)
[0 2 3 1 3 1 0]
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [1. 0. 0. 0.]]
['a']


/Users/rcperez/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [83]:

#SeqIO.parse(open(gb_file,"r"), "gb")

#for record in SeqIO.parse("NC_001133", "gb"):
    #print(record)

In [84]:
from Bio import Entrez


Entrez.email = "rcperez@stanford.edu"
search_term = "Schizophyllum+commune[organism] AND genbank[filter] AND "
handle = Entrez.esearch(db="nucleotide", term=search_term)
genome_ids = Entrez.read(handle)['IdList']

for genome_id in genome_ids:
    record = Entrez.efetch(db="nucleotide", id=genome_id, rettype="gbwithparts", retmode="text")
    filename = 'GenBank_Record_{}.gbk'.format(genome_id)
    #print('Writing:{}'.format(filename))
    with open(filename, "w") as f:
        f.write(record.read())
#print(genome_ids)

In [21]:
from Bio import Entrez

In [25]:
def get_assembly_summary(id):
    """Get esummary for an entrez id"""
    from Bio import Entrez
    esummary_handle = Entrez.esummary(db="assembly", id=id, report="full")
    esummary_record = Entrez.read(esummary_handle)
    return esummary_record

def get_assemblies(term, download=True, path='assemblies'):
    """Download genbank assemblies for a given search term.
    Args:
        term: search term, usually organism name
        download: whether to download the results
        path: folder to save to
    """

    
    #provide your own mail here
    Entrez.email = "rcperez@stanford.edu"
    handle = Entrez.esearch(db="assembly", term=term, retmax='200')
    record = Entrez.read(handle)
    ids = record['IdList']
    #print (f'found {len(ids)} ids')
    links = []
    for id in ids:
        #get summary
        summary = get_assembly_summary(id)
        #get ftp link
        url = summary['DocumentSummarySet']['DocumentSummary'][0]['FtpPath_RefSeq']
        if url == '':
            continue
        label = os.path.basename(url)
        #get the fasta link - change this to get other formats
        link = os.path.join(url,label+'_genomic.fna.gz')
        print (link)
        links.append(link)
        #if download == True:
            #download link
            #urllib.request.urlretrieve(link, f'{label}.fna.gz')
    return links

In [27]:
get_assemblies('Schizophyllum+commune[organism] AND genbank[filter]')

[]